In [8]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large as ml
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Concatenate,Lambda

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# izlenecek metriği belirtir,argümanı, gelişme olmadığı durumda kaç epoh sonra eğitimin durdurulacağını belirtir ,
#eğitimin sonunda en iyi ağırlıkların geri yüklenecek

In [62]:
conv_base=tf.keras.applications.VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
conv_base.summary()
conv_base.trainable=True
set_trainable=False

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_77 (InputLayer)          │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [72]:
for layer in conv_base.layers:
    if layer.name=='block_conv3':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False

In [73]:
def fire_module(inputs, squeeze_depth, expand_depth):
    squeezed = tf.keras.layers.Conv2D(filters=squeeze_depth, kernel_size=1, activation='relu', padding='same')(inputs)
    expanded1x1 = tf.keras.layers.Conv2D(filters=expand_depth, kernel_size=1, activation='relu', padding='same')(squeezed)
    expanded3x3 = tf.keras.layers.Conv2D(filters=expand_depth, kernel_size=3, activation='relu', padding='same')(squeezed)
    output = tf.keras.layers.Concatenate()([expanded1x1, expanded3x3])
    return output

def SqueezeNet(input_shape=(224, 224, 3), num_classes=2):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(filters=96, kernel_size=7, strides=2, activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    x = fire_module(x, squeeze_depth=16, expand_depth=64)
    x = fire_module(x, squeeze_depth=16, expand_depth=64)
    x = fire_module(x, squeeze_depth=32, expand_depth=128)
    x = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    x = fire_module(x, squeeze_depth=32, expand_depth=128)
    x = fire_module(x, squeeze_depth=48, expand_depth=192)
    x = fire_module(x, squeeze_depth=48, expand_depth=192)
    x = fire_module(x, squeeze_depth=64, expand_depth=256)
    x = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    x = fire_module(x, squeeze_depth=64, expand_depth=256)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=1, activation='relu', padding='same')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    output = tf.keras.layers.Activation('softmax')(x)

    model = tf.keras.models.Model(inputs, output)
    return model

model = SqueezeNet()
model.summary()

Model: "functional_40"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_79 (InputLayer)   │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_414 (Conv2D)           │ (None, 112, 112, 96)      │          14,208 │ input_layer_79[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_48              │ (None, 56, 56, 96)        │               0 │ conv2d_414[0][0]           │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_415 (Conv2D)           │ (None, 56, 56, 16)        │           1,552 │ max_pooling2d_48[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_416 (Conv2D)           │ (None, 56, 56, 64)        │           1,088 │ conv2d_415[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_417 (Conv2D)           │ (None, 56, 56, 64)        │           9,280 │ conv2d_415[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_143 (Concatenate) │ (None, 56, 56, 128)       │               0 │ conv2d_416[0][0],          │
│                               │                           │                 │ conv2d_417[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_418 (Conv2D)           │ (None, 56, 56, 16)        │           2,064 │ concatenate_143[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_419 (Conv2D)           │ (None, 56, 56, 64)        │           1,088 │ conv2d_418[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_420 (Conv2D)           │ (None, 56, 56, 64)        │           9,280 │ conv2d_418[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_144 (Concatenate) │ (None, 56, 56, 128)       │               0 │ conv2d_419[0][0],          │
│                               │                           │                 │ conv2d_420[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_421 (Conv2D)           │ (None, 56, 56, 32)        │           4,128 │ concatenate_144[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_422 (Conv2D)           │ (None, 56, 56, 128)       │           4,224 │ conv2d_421[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_423 (Conv2D)           │ (None, 56, 56, 128)       │          36,992 │ conv2d_421[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_145 (Concatenate) │ (None, 56, 56, 256)       │               0 │ conv2d_422[0][0],          │
│                               │                           │                 │ conv2d_423[0][0]           │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 736,450 (2.81 MB)

 Trainable params: 736,450 (2.81 MB)

 Non-trainable params: 0 (0.00 B)

In [74]:
resize_layer = tf.keras.layers.Conv2D(3, (1, 1), activation='relu')

In [114]:
#model=tf.keras.models.Sequential()
model_bir= tf.keras.Sequential([
    conv_base,
    resize_layer,
    tf.keras.layers.UpSampling2D(size=(32, 32)),
    model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [115]:
model_bir.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5),
           loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
 #  
# model.summary()

In [116]:
egitim2=r'C:\Users\06bey\kolay\train'
val2=r'C:\Users\06bey\kolay\val'
test2=r'C:\Users\06bey\kolay\test'

In [117]:
#veri arttırma
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
rescale=1./255,#px normalizasyon
    rotation_range=40,#döndürme işlemi
    width_shift_range=0.2,#yatay kaydırma
    height_shift_range=0.2,
    shear_range=0.2, 
    zoom_range=0.2,#yakınlaş
    horizontal_flip=True,#aynalama
    fill_mode='nearest'#eğer px oluşursa 'nearest'ile yakınındaki px. rengi ile doldur
)

In [118]:
train_generator=train_datagen.flow_from_directory(
    egitim2, 
    target_size=(224,224),
    batch_size=32
)

Found 1083 images belonging to 2 classes.


In [119]:
val_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255 
)

In [120]:
val_generator=val_datagen.flow_from_directory(
    val2, 
    target_size=(224,224),
    batch_size=32,   
)

Found 201 images belonging to 2 classes.


In [121]:
egit = model_bir.fit(
    train_generator,
    steps_per_epoch=16,
    epochs=7,
    #callbacks=[early_stopping],#kaynak kullanımını optimize etmek için
    validation_data=val_generator,
    validation_steps=1
)

Epoch 1/7


ValueError: Input 0 of layer "functional_40" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, 7168, 7168, 512)

In [37]:
from tensorflow.keras.applications import MobileNetV3Small

mobilenetv3_small = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
mobilenetv3_small.trainable = False

input_layer = Input(shape=(224, 224, 3))
mobilenetv3_small_output = mobilenetv3_small(input_layer)
x = tf.keras.layers.Flatten()(mobilenetv3_small_output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
output_layer = tf.keras.layers.Dense(2, activation='softmax')(x)
combined_model = Model(inputs=input_layer, outputs=output_layer)

model_bir= tf.keras.Sequential([
    combined_model,
    model, 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model_bir.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

# Modeli eğit
egit = model_bir.fit(
    train_generator,
    steps_per_epoch=16,
    epochs=5,
    validation_data=val_generator,
    validation_steps=1
)


Epoch 1/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5090 - loss: 0.6931 - val_accuracy: 0.4062 - val_loss: 0.6935
Epoch 2/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 704ms/step - accuracy: 0.4454 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.3730 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 971ms/step - accuracy: 0.4751 - loss: 0.6932 - val_accuracy: 0.5312 - val_loss: 0.6931
Epoch 5/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 12s 735ms/step - accuracy: 0.5459 - loss: 0.6931 - val_accuracy: 0.5312 - val_loss: 0.6930


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large, MobileNetV3Small
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import BinaryCrossentropy

mobilenetv3_large = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
mobilenetv3_large.trainable = False

mobilenetv3_small = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
mobilenetv3_small.trainable = False

# Input layer 
input_layer = Input(shape=(224, 224, 3))

# Small MobileNetV3 modelinden çıkan çıktıyı al
mobilenetv3_small_output = mobilenetv3_small(input_layer)


x = Flatten()(mobilenetv3_small_output)

# İki yoğun katman ekle
x = Dense(256, activation='relu')(x)
output_layer = Dense(2, activation='softmax')(x)

# Combined modeli oluştur
combined_model = Model(inputs=input_layer, outputs=output_layer)

# İkinci bir model oluştur
model_b = tf.keras.Sequential([
    combined_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')
])

# Modeli derle
model_b.compile(optimizer=RMSprop(learning_rate=1e-5),
              loss=BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

egit = model_b.fit(
    train_generator,
    steps_per_epoch=16,
    epochs=5,
    validation_data=val_generator,
    validation_steps=1
)


Epoch 1/5


In [38]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate
from tensorflow.keras.applications import MobileNetV3Large

# MobileNetV3Large modelini oluştur
mobilenetv3_large = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# MobileNetV3Large modelini dondur
mobilenetv3_large.trainable = False

# İlk giriş katmanını tanımla
input_layer = Input(shape=(224, 224, 3))

# MobileNetV3Large modelinin çıktısını al
mobilenetv3_large_output = mobilenetv3_large(input_layer)

# Çıktıyı düzleştir
x = Flatten()(mobilenetv3_large_output)

# Yoğun katmanı ekle
x = Dense(256, activation='relu')(x)

# SqueezeNet için giriş katmanını tanımla
squeezenet_input = Input(shape=(224, 224, 3))

# SqueezeNet modelinden çıktıyı al
squeezenet_output = model(squeezenet_input)

# Çıktıları birleştir
concatenated_output = Concatenate()([x, squeezenet_output])

# Çıkış katmanını oluştur
output_layer = Dense(2, activation='softmax')(concatenated_output)

# Kombine modeli oluştur
combined_model = Model(inputs=[input_layer, squeezenet_input], outputs=output_layer)

# Modeli derle
combined_model.compile(optimizer='rmsprop',
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

# Eğitim ve doğrulama verilerini hazırla
x_train, y_train = next(train_generator)
x_val, y_val = next(val_generator)

# Modeli eğit
egit = combined_model.fit(
    x=[x_train, x_train],  # Eğitim verisi
    y=y_train,  # Eğitim etiketleri
    steps_per_epoch=16,
    epochs=15,
    validation_data=([x_val, x_val], y_val),  # Doğrulama verisi ve etiketleri
    validation_steps=16  # Doğrulama adımları
)


Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - accuracy: 0.5582 - loss: 0.0000e+00WARNING:tensorflow:5 out of the last 20 calls to <function TensorFlowTrainer.make_test_function.<locals>.one_step_on_iterator at 0x00000287EF0DAFC0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 537ms/step - accuracy: 0.5566 - loss: 0.0000e+00 - val_accuracy: 0.5312 - val_loss: 0.0000e+00
Epoch 2/15


AttributeError: 'NoneType' object has no attribute 'items'